![openclassrooms](https://s3.eu-west-1.amazonaws.com/course.oc-static.com/courses/6204541/1+HnqdJ-5ofxiPP9HIxdNdpw.jpeg)

# Filtrez les données du dataframe

Nous allons à présent aller un peu plus loin dans la manipulation de nos prêts bancaires. Les demandes suivantes ont été formulées par le responsable du service prêt de notre établissement :
- Le taux d’endettement autorisé est de 35%. Pourriez vous me communiquer le nombre de personnes ayant dépassé ce seuil ?
- Même question, mais cette fois ci uniquement sur l’agence Parisienne
- Pour faciliter le traitement d’éventuelles futures demandes de prêts, pourriez vous ajouter une variable nommée `risque` qui nous permettrait d’identifier les clients risqués facilement
- Combien de prêts automobiles ont été accordés ? Quel est le coût total moyen de ces derniers ?
- Quel est le bénéfice mensuel total réalisé par l’agence Toulousaine ?

Vous allez devoir mettre en application l’ensemble des processus de sélection présentés ci-dessus pour répondre à ces différentes demandes.

In [1]:
import numpy as np
import pandas as pd

In [2]:
# traitement réalisés précédemment
prets = pd.read_csv('https://raw.githubusercontent.com/OpenClassrooms-Student-Center/fr-4452741-decouvrez-les-librairies-python-pour-la-data-science/main/data/prets.csv')

# calcul du taux d'endettement
prets['taux_endettement'] = round(prets['remboursement'] * 100 / prets['revenu'], 2)

# renommer taux en taux_interet
prets.rename(columns={'taux':'taux_interet'}, inplace=True)

# calculer le cout total du pret
prets['cout_total'] = prets['remboursement'] * prets['duree']

# calculer les bénéfices mensuels réalisés
prets['benefices'] = round((prets['cout_total'] * prets['taux_interet']/100)/(24), 2)

prets.head()

,identifiant,ville,CP,revenu,remboursement,duree,type,taux_interet,taux_endettement,cout_total,benefices
0,0,TOULOUSE,31100,3669.0,1130.05,240,immobilier,1.168,30.80,271212.0,131.99
1,1,PARIS,75009,5310.0,240.00,64,automobile,3.701,4.52,15360.0,23.69
2,1,PARIS,75009,5310.0,1247.85,300,immobilier,1.173,23.50,374355.0,182.97
3,2,MARSEILLE,13010,1873.0,552.54,240,immobilier,0.972,29.50,132609.6,53.71
4,3,MARSEILLE,13010,1684.0,586.03,180,immobilier,1.014,34.80,105485.4,44.57


Le taux d’endettement autorisé est de 35%. Pourriez vous me communiquer le nombre de personnes ayant dépassé ce seuil ? vous pourrez me stocker la liste dans une variable à part nommé `clients_risque`

In [33]:
nb_pers_endette = prets.loc[prets['taux_endettement'] > 35, :].shape[0]
print("Il y a", nb_pers_endette, "personnes ayant dépassé le taux d'endettement autorisé")

Il y a 58 personnes ayant dépassé le taux d'endettement autorisé


Même question, mais cette fois ci uniquement sur l’agence Parisienne

In [32]:
nb_pers_endette_paris = prets.loc[(prets['taux_endettement'] > 35) & (prets['ville'] == 'PARIS'), :].shape[0]
print("Il y a", nb_pers_endette_paris, "personnes de l'agence parisienne ayant dépassé le taux d'endettement autorisé")

Il y a 16 personnes de l'agence parisienne ayant dépassé le taux d'endettement autorisé


Pour faciliter le traitement d’éventuelles futures demandes de prêts, pourriez vous ajouter une variable nommée `risque` qui aurait pour valeur `Oui` si le client est risqué (taux d’endettement > 35%), `Non` sinon.
Pour se faire, je vous conseille de créer la variable `risque` en l'initialisant à `Non` (ou `Oui`, à votre préférence) et de ne modifier que les lignes concernées par la valeur inverse :

In [12]:
prets["risque"] = "Oui"
prets.loc[prets["taux_endettement"] < 35, "risque"] = "Non"
prets

,identifiant,ville,CP,revenu,remboursement,duree,type,taux_interet,taux_endettement,cout_total,benefices,risque
0,0,TOULOUSE,31100,3669.0,1130.05,240,immobilier,1.168,30.80,271212.0,131.99,Non
1,1,PARIS,75009,5310.0,240.00,64,automobile,3.701,4.52,15360.0,23.69,Non
2,1,PARIS,75009,5310.0,1247.85,300,immobilier,1.173,23.50,374355.0,182.97,Non
3,2,MARSEILLE,13010,1873.0,552.54,240,immobilier,0.972,29.50,132609.6,53.71,Non
4,3,MARSEILLE,13010,1684.0,586.03,180,immobilier,1.014,34.80,105485.4,44.57,Non
...,...,...,...,...,...,...,...,...,...,...,...,...
239,224,PARIS,75008,5118.0,1248.79,300,immobilier,1.206,24.40,374637.0,188.26,Non
240,224,PARIS,75008,5118.0,238.20,25,automobile,3.423,4.65,5955.0,8.49,Non
241,225,BORDEAUX,33100,2356.0,876.43,180,immobilier,1.172,37.20,157757.4,77.04,Oui
242,226,PARIS,75002,5098.0,2910.96,240,immobilier,1.140,57.10,698630.4,331.85,Oui


Combien de prêts automobiles ont été accordés ? Quel est le coût total moyen de ces derniers ?

In [37]:
mask = prets['type'] == "automobile"
count_auto = prets.loc[mask].shape[0]
print("Il y a eu", count_auto, "prets automobiles")
cout_mean_auto = round(prets.loc[mask, 'cout_total'].mean() / 1000, 2)
print("Le coût total des prets automobiles s'élèvent à", cout_mean_auto, "k€")

Il y a eu 16 prets automobiles
Le coût total des prets automobiles s'élèvent à 13.66 k€


Quel est le bénéfice mensuel total réalisé par l’agence Toulousaine ?

In [25]:
benefice_toulouse = round(prets.loc[prets['ville'] == "TOULOUSE", "benefices"].sum())
print("Le bénéfice mensuel réalisé par l'agence de Toulouse est de", benefice_toulouse, "€")

Le bénéfice mensuel réalisé par l'agence de Toulouse est de 2138 €


Félicitation ! Je vous invite fortement à vérifier vos réponses avec [la correction ci contre](https://colab.research.google.com/github/OpenClassrooms-Student-Center/fr-4452741-decouvrez-les-librairies-python-pour-la-data-science/blob/main/notebooks/P2/P2C3%20-%20Filtrez%20les%20donn%C3%A9es%20du%20data%20frame%20-%20correction.ipynb).